# $^{83m}Kr$ LCE Maps: Xerawdp vs. Pax
_Ted Berger, Dan Alexander; June 15 2016_

# Table of Contents
[1 - Introduction](#introduction)  
[2 - Data and Event Selection](#data_and_selection)  

[? - Next Steps](#next_steps)  
[?? - References](#references) 

<a id='introduction'></a>
# Introduction

As an internal calibration, $^{83m}Kr$ proves very useful in a variety of liquid noble detectors. We utilize a solution of $^{83}Rb$ evaporated onto zeolite beads to generate $^{83m}Kr$ with a half-life of $86.2\ days$. From there, $^{83m}Kr$, with a half-life of $1.83\ hr$, generates a $32.1\ keV$ conversion electron or gamma with an internal conversion coefficient of $\alpha=2035$. This is then followed by a $9.4\ keV$ conversion electron or gamma with an internal conversion coefficient of $\alpha=17.09$. These decays occur with a delay half-life of $154.4\ ns$. This decay scheme is shown in Figure 0.

<img src="kr83m_decay_scheme.png">
$$\textbf{Figure 0: }^{83m}Kr\ \text{ Decay Schematic} $$

The short half-life of $^{83m}Kr$ and its mother permit an internal calibration without introducing unwanted signals into the experiment. Furthermore, the short half-life between the $32.1\ keV$ and $9.4\ keV$ calibration lines provides a unique signal that is essentially background free.

This calibration should provide a uniform distribution of events throughout the detector allowing the analysis of position reconstruction algorithms, non-uniformities in drift field, relative quantum efficiencies of PMTs, light collection efficiency maps, and absolute light yield. Furthermore, the XENON100 detector is capable of measuring the half-life of the $^{83m}Kr_{1/2-}$ and $^{83m}Kr_{7/2+}$ states, though the nuances of time smearing are still under investigation. 

This note focusses on understanding the $^{83m}Kr$ data and generating LCE maps with the intent of building these maps for XENON1T (hopefully very soon!).

<a id='data_and_selection'></a>
# Data and Event Selection

In [1]:
from xerawdp_helpers import *    # helper functions for retrieving xerawdp data
from Kr83m_Basic import *        # pax minitree class for Kr83m data

import hax
hax.init(main_data_paths=['/project/lgrandi/xenon100/archive/root/merged/xenon100/run_14_pax4.1.2/'],
         raw_data_local_path='/project/lgrandi/tunnell/')

#hax.ipython.code_hider()

/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# from 2015 Kr83m diffusion-mode data
datasets_xerawdp = ['xe100_150413_1839','xe100_150414_1535','xe100_150415_1749',
            'xe100_150416_1832','xe100_150419_1611','xe100_150420_0304',
            'xe100_150420_1809']

# xerawdp path
xerawdpPath = '/project/lgrandi/tunnell/run_14/NewNN/'

# get xerawdp tree, apply event restrictions, and retrieve desirable data
#  see xerawdp_helpers.py for details
xerawdpTree = load_xerawdp_tree(datasets_xerawdp[:1], xerawdpPath)
df_xerawdp = build_xerawdp_df(xerawdpTree)
df_xerawdp.to_pickle('xerawdp_2S1and1S2.pkl')

In [ ]:
df_xerawdp = pd.read_pickle('xerawdp_2S1and1S2.pkl')

In [ ]:
# datasets processed by pax_4.1.2
datasets_pax = ['xe100_150413_1839','xe100_150414_1535',
               'xe100_150419_1611','xe100_150420_0304',
               'xe100_150420_1809']

# load minitrees
# throws error when building minitrees for many datasets, I built them one by one

data = hax.minitrees.load(datasets_pax, treemakers=Kr83m_Basic)
data = data[data['s10Time']>=0] # remove NaNs

<a id='references'></a>
# References

[[1] S. -C. Wu, Nuclear Data Sheets 92, 893 (2001)](nucleardatasheets_a83.pdf)  